In [ ]:
#| default_exp core

## tableform
> `_repr_html_` for nested data

In [ ]:
#|export
import warnings
import sys
from numbers import Integral

In [ ]:
#|export
def _get_or_last(l, i):
    try: return l[i]
    except:
        try: return l[-1]
        except: return l

In [ ]:
#|export
def _is_astropy_quantity(a):
    return str(a.__class__) == "<class 'astropy.units.quantity.Quantity'>"

In [ ]:
#|export
def _is_pint_quantity(a):
    return str(a.__class__) == "<class 'pint.Quantity'>"

In [ ]:
#|export
def _package(obj): return obj.__class__.__module__.split('.')[0]

In [ ]:
#|export
bracket_class = "table-form-bracketed"
boxed_class = "table-form-boxed"
empty_string = ''

In [ ]:
#|export
class TableForm:
    def __init__(self, a=None, fmt='.2f', border_color="Gray", width="auto", edgeitems='auto', font_size=12, padding=2):
        self.a = a
        self.fmt = fmt
        if border_color == 'auto':
            border_color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
        elif isinstance(border_color, str):
            border_color = [border_color]
        self.border_color = border_color
        self.width = width
        self.font_size = font_size
        self.padding = padding
        self.edgeitems = edgeitems
        self.units = None

    def table_tag(self, content, depth, brackets=False):
        return (
            (f'<div style="font-size:{self.font_size}px">{self.table_caption()}</div>' if depth == 0 else '') + 
            f'<table data-quarto-disable-processing="true" class="table-form {bracket_class if brackets else boxed_class}" style="display:inline-table; text-align:center;' +
            self.font_style() +
            self.table_width(depth) +
            self.table_border(depth) +
            (self.table_brackets() if brackets else "") +
            self.table_padding() +
            self.table_margin(depth) +
            '">' +
            content +
            '</table>'
        )

    def table_caption(self):
        caption = f'{_package(self.a)} {self.a.__class__.__name__}'
        try: caption += f' {tuple(self.a.shape)} {self.a.dtype}'
        except: pass
        if self.units is not None: caption += f' {self.units}'
        return caption
            
    def table_padding(self):
        return f'padding: {self.padding}px;'
        
    def font_style(self):
        return f'font-size:{self.font_size}px;'
        
    def table_width(self, depth):
        return f'width: {_get_or_last(self.width, depth)};'

    def table_border(self, depth):
        return f'border: 1.5px solid {self.border_color[depth % len(self.border_color)]};'

    def table_brackets(self):
        return (
            "border-top-style: none;" +  
            "border-bottom-style: none;" +
            "border-radius: 5px;" +
            "border-spacing: 0px;" +
            "border-collapse:separate;" 
        )

    def table_margin(self, depth):
        return f'margin: 0px {"0" if depth == 0 else "auto"};'
    
    def caption_tag(self, content):
        return f'<caption style="white-space: nowrap">{content}</caption>'
    
    def td_tag(self, content):
        return f'<td style="padding: {self.padding}px; text-align: center; white-space: nowrap;">{content}</td>'
    
    def tr_tag(self, content):
        return f'<tr style="padding: 0; margin: 0">{content}</tr>'
    
    def abbreviate(self, a, symbol='…'):
        try: # For some reason numpy masked arrays throw error here, otherwise wouldn't need try block
            if a[0] in ['…', '︙']: return a
        except ValueError: pass
        l = self._edgeitems
        if l and len(a) > 2 * l + 1:
            head = a[:l]
            tail = a[-l:]
            try: 
                if len(a[l - 1]) <= 2 * l + 1:
                    ellipses = [symbol] * len(a[l - 1])
                else:
                    ellipses = [symbol] * l + ["⋱"] + [symbol] * l
            except: ellipses = symbol
            return [*head, ellipses, *tail]
        return a
    
    def make_entry(self, d, depth=0):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            try:
                if d == '⋱': return d
                if set(d) <= {'…', '⋱'}: return '…'
            except: pass
            try: return self.make_table(d, depth+1)
            except:
                if isinstance(d, Integral): return d
                try: return format(d, self.fmt)
                except: return d
    
    def make_row(self, a, depth=0):
        if isinstance(a, str): raise TypeError
        a = self.abbreviate(a, '…')
        return self.tr_tag(''.join(self.td_tag(self.make_entry(entry, depth)) for entry in a))
    
    def make_rows(self, a, depth=0):
        a = self.abbreviate(a, "︙")
        return "\n".join(self.make_row(row, depth) for row in a)
    
    def make_table(self, a, depth=0):
        if isinstance(a, str): raise TypeError
        if hasattr(a, 'ndim') and a.ndim == 0: return self.table_tag(self.make_entry(a.item(), depth), depth, brackets=False) 
        try:
            return self.table_tag(self.make_rows(a, depth), depth, brackets=False)
        except:
            return self.table_tag(self.make_row(a, depth), depth, brackets=True) 

    def resolve_edgeitems(self):
        if self.edgeitems == 'auto':
            try:
                if self.a.ndim > 3: self._edgeitems = 1
                elif self.a.ndim == 3: self._edgeitems = 2
                else: self._edgeitems = 3
            except AttributeError: self._edgeitems = 3
        else: self._edgeitems = self.edgeitems
            
    def _repr_html_(self):
        # if hasattr(self.a, 'ndim') and self.a.ndim == 0: return self.a._repr_html_()
        self.resolve_edgeitems()
        value = self.a
        if _is_pint_quantity(self.a):
            self.units = f'{value.units:~P}'
            value = value.magnitude
        if _is_astropy_quantity(self.a): 
            self.units = value.unit.to_string('unicode')
            value = value.value
            # return f'<table><tr><td>{self.make_entry(mag, depth=-1)}</td><td>{units}</td></tr></table>'
        return self.make_entry(value, depth=-1)

    @classmethod
    def register(cls, *types):
        """Register `types` in IPython to display using `TableForm`."""
        try: 
            formatter = get_ipython().display_formatter.formatters['text/html']
            for _type in types: formatter.for_type(_type, lambda o: cls(o)._repr_html_())
        except NameError: pass

#### lists and tuples

In [ ]:
TableForm([1, 2])

1,2


In [ ]:
TableForm([[1, 2]])

1,2


In [ ]:
TableForm([[[1, 2]]])

1,2


In [ ]:
TableForm([1, [1, [[1, 2, [[1], 2]]]]])

In [ ]:
TableForm((1, [1, [(1, 2, [[1], 2])]]))

#### numpy arrays

In [ ]:
import numpy as np

In [ ]:
TableForm(np.arange(2))

0,1


In [ ]:
TableForm([1, [1, [[1, 2, [[1], 2]]]]], border_color='auto', padding=10)

In [ ]:
TableForm(5 * np.arange(4 * 5 * 4 * 3 * 4).reshape(4, 5, 4, 3, 4), edgeitems=1, border_color=['red', 'green', 'blue'])

`'auto'` setting for `edgeitems`

In [ ]:
TableForm(np.arange(6))

0,1,2,3,4,5


In [ ]:
TableForm(np.arange(6**2).reshape(6, 6))

0,1,2,3,4,5
6,7,8,9,10,11
12,13,14,15,16,17
18,19,20,21,22,23
24,25,26,27,28,29
30,31,32,33,34,35


In [ ]:
TableForm(np.arange(6**3).reshape([6] * 3))

In [ ]:
TableForm(np.arange(6**4).reshape([6] * 4))

In [ ]:
TableForm(np.arange(6**5).reshape([6] * 5))

In [ ]:
TableForm(np.arange(6**6).reshape([6] * 6))

Complex dtype

In [ ]:
TableForm(np.random.normal(size=(40, 40, 40)) + 1j * np.random.normal(size=(40, 40, 40)), edgeitems=3)

#### pint `Quantity`s

In [ ]:
import pint

In [ ]:
u = pint.UnitRegistry(force_ndarray_like=True)

In [ ]:
TableForm(2.54 * u.cm)

In [ ]:
TableForm(np.ones((3, 4, 5)) * u.m**2)

1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00


#### astropy `Quantity`s

In [ ]:
import astropy.units as apu

In [ ]:
TableForm(apu.Quantity([1., 1.] , unit=apu.m**2))

1.00,1.00


In [ ]:
TableForm(apu.Quantity(1., unit=apu.m**2))

#### Register formatter

In [ ]:
#|export
try: TableForm.register(sys.modules['numpy'].ndarray, sys.modules['pint'].Quantity)
except KeyError: pass

In [ ]:
np.ma.array(np.random.rand(4, 4), mask=np.random.randint(2, size=[4, 4]))

0.00,0.56,0.00,0.22
0.00,0.44,0.20,0.00
0.00,0.91,0.00,0.00
0.00,0.00,0.00,0.00


In [ ]:
np.ones((3, 4, 5)) * u.m**2

1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00
1.00,1.00,1.00,1.00,1.00


In [ ]:
apu.Quantity([1., 1.], unit=apu.m**2)

1.00,1.00


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()